## Import Libraries

In [1]:
import os
import webbrowser
from neo4j import GraphDatabase
from pyvis.network import Network
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time

## Load Data

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/movie-config.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Perform Queries

In [5]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
with driver.session() as session:
    result = session.run(
        """
        MATCH (n:Actor) RETURN n LIMIT 25
        """
    )

    # Display the results
    for record in result:
        actor = record['n']
        print(f"Actor: {actor['name']}")
        print("-" * 40)

driver.close()

Actor: François Lallement
----------------------------------------
Actor: Jules-Eugène Legris
----------------------------------------
Actor: Lillian Gish
----------------------------------------
Actor: Mae Marsh
----------------------------------------
Actor: Henry B. Walthall
----------------------------------------
Actor: Miriam Cooper
----------------------------------------
Actor: Dan Hanlon
----------------------------------------
Actor: Edna Pendleton
----------------------------------------
Actor: Curtis Benton
----------------------------------------
Actor: Allen Holubar
----------------------------------------
Actor: Spottiswoode Aitken
----------------------------------------
Actor: Mary Alden
----------------------------------------
Actor: Frank Bennett
----------------------------------------
Actor: Edna Purviance
----------------------------------------
Actor: Eric Campbell
----------------------------------------
Actor: Albert Austin
-------------------------------------

In [7]:
with driver.session() as session:
    result = session.run(
        """
        MATCH (m:Movie)-[:ACTED_IN]-(a:Actor)
        WHERE a.name = 'Tom Hanks'
        UNWIND m.languages AS lang
        RETURN m.title AS movie,
               m.languages AS languages,
               lang AS language
        """
    )

    # Display the results
    for record in result:
        print(f"Movie: {record['movie']}")
        print(f"Languages: {record['languages']}")
        print(f"Language: {record['language']}")
        print("-" * 40)

driver.close()

C:\Users\haoch\AppData\Local\Temp\ipykernel_1744\3786384549.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Movie: Punchline
Languages: ['English']
Language: English
----------------------------------------
Movie: Catch Me If You Can
Languages: ['English', ' French']
Language: English
----------------------------------------
Movie: Catch Me If You Can
Languages: ['English', ' French']
Language:  French
----------------------------------------
Movie: Dragnet
Languages: ['English']
Language: English
----------------------------------------
Movie: Saving Mr. Banks
Languages: ['English']
Language: English
----------------------------------------
Movie: Bachelor Party
Languages: ['English', ' Japanese']
Language: English
----------------------------------------
Movie: Bachelor Party
Languages: ['English', ' Japanese']
Language:  Japanese
----------------------------------------
Movie: Volunteers
Languages: ['English']
Language: English
----------------------------------------
Movie: Man with One Red Shoe, The
Languages: ['English']
Language: English
----------------------------------------
Movie:

## Node visualisation

In [8]:
# Create a Pyvis network for visualization
net = Network(height="750px", width="100%", directed=True)
net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=200, spring_strength=0.08, damping=0.4)

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
    result = session.run(
        """
        MATCH (m:Movie)-[:ACTED_IN]-(a:Actor)
        WHERE a.name = 'Tom Hanks'
        UNWIND m.languages AS lang
        RETURN a.name AS actor,
               m.title AS movie,
               lang AS language
        """
    )

    for record in result:
        actor = record["actor"]
        movie = record["movie"]
        language = record["language"]

        print(f"Actor: {actor}, movie: {movie}, language: {language}.")

        # Add nodes
        net.add_node(actor, label=actor, color='lightblue', shape='dot')
        net.add_node(movie, label=movie, color='orange', shape='box')
        net.add_node(language, label=language, color='lightgreen', shape='ellipse')

        # Add edges
        net.add_edge(actor, movie, label="ACTED_IN")
        net.add_edge(movie, language, label="language")

# Save and show the graph
path = "../../Playground/output/tom_hanks_movies.html"
net.write_html(path)

# Take screenshot with Selenium
screenshot_path = "../../Playground/output/images/tom_hanks_movies.png"

# Set up headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1200,800")
driver_path = "chromedriver"  # Assumes it's in your PATH
browser = webdriver.Chrome(options=chrome_options)

# Open file URL
file_url = "file://" + os.path.abspath(path)
browser.get(file_url)
time.sleep(2)  # Wait for graph to load

# Save screenshot
browser.save_screenshot(screenshot_path)
browser.quit()

driver.close()

Actor: Tom Hanks, movie: Punchline, language: English.
Actor: Tom Hanks, movie: Catch Me If You Can, language: English.
Actor: Tom Hanks, movie: Catch Me If You Can, language:  French.
Actor: Tom Hanks, movie: Dragnet, language: English.
Actor: Tom Hanks, movie: Saving Mr. Banks, language: English.
Actor: Tom Hanks, movie: Bachelor Party, language: English.
Actor: Tom Hanks, movie: Bachelor Party, language:  Japanese.
Actor: Tom Hanks, movie: Volunteers, language: English.
Actor: Tom Hanks, movie: Man with One Red Shoe, The, language: English.
Actor: Tom Hanks, movie: Splash, language: English.
Actor: Tom Hanks, movie: Splash, language:  Swedish.
Actor: Tom Hanks, movie: Big, language: English.
Actor: Tom Hanks, movie: Nothing in Common, language: English.
Actor: Tom Hanks, movie: Money Pit, The, language: English.
Actor: Tom Hanks, movie: Money Pit, The, language:  Spanish.
Actor: Tom Hanks, movie: Money Pit, The, language:  Chinese.
Actor: Tom Hanks, movie: Toy Story of Terror, langu